# Dot Product Test

The dot product test is a validation tool used primarily in the realm of computational algorithms, especially those associated with solving linear systems or optimization problems.

## Concept

Given two operators \( A \) and \( A^* \) (where \( A^* \) is the adjoint of \( A \)), and two random vectors \( x \) and \( y \), the dot product test checks the following equality:

\[ \langle Ax, y \rangle = \langle x, A^*y \rangle \]

Where:
- \( \langle ., . \rangle \) denotes the dot product.
- \( Ax \) is the result of applying operator \( A \) to vector \( x \).
- \( A^*y \) is the result of applying the adjoint of operator \( A \) to vector \( y \).

## Significance

If the above equality holds true for various random vectors \( x \) and \( y \), it's a strong indication that the implemented adjoint (or transposed) operator \( A^* \) is correct. This test is particularly useful in iterative methods where both the operator and its adjoint are used, and correctness is essential for convergence.

## Steps

1. Choose random vectors \( x \) and \( y \).
2. Compute \( \langle Ax, y \rangle \) and \( \langle x, A^*y \rangle \).
3. Compare the two results. They should be equal (or very close, considering computational precision).

Remember, a successful dot product test usually implies the adjoint operator is correctly implemented, but it doesn't validate the correctness of the primary operator \( A \).


In [ ]:
%load_ext autoreload
%autoreload 2
import sys

! pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@3fac86108f59c822193cbd6f28687fecce5e298b" 
import generic_solver


In [24]:



def test_dot(oper):
    mod1=oper.domain.clone()
    mod2=mod1.clone()
    dat1=oper.range.clone()
    dat2=dat1.clone()

    mod1.rand()
    dat1.rand()

    oper.forward(False,mod1,dat2)
    oper.adjoint(False,mod2,dat1)

    dot1=mod1.dot(mod2)
    dot2=dat1.dot(dat2)
    if abs((dot1-dot2)/dot2 > 1e-6):
        print(dot1,dot2)
        raise Exception("Failed product false add=False")
    oper.forward(True,mod1,dat2)
    oper.adjoint(True,mod2,dat1)

    dot1=mod1.dot(mod2)
    dot2=dat1.dot(dat2)

    if abs((dot1-dot2)/dot2 > 1e-6):
        print(dot1,dot2)
        raise Exception("Failed dot product false add=True")

In [31]:
class BoxcarF(Operator):

    def __init__(self, mod, dat,halflen):
        """
        Initialize a boxcar convolution (smoothing)

            mod, dat - sepVector
            halflen - Half length of smoothing box
        """
        super().__init__(mod, dat)
        if not isinstance(mod, FloatVector) or not isinstance(dat,FloatVector):
            raise Exception("Expecting model, data, flt to be sepVectors")
        self._halflen=halflen
        self._nd=dat.get_hyper().axes[0].n


    def forward(self, add, mod, dat):
        """
        Forward operation
        """
        self.checkDomainRange(mod, dat)
        if not add:
            dat.zero()
        sc=1./(1+2.*self._halflen)
        for i in range(self._nd):
            tmp=0
            for ib in range(i-self._halflen,i+self._halflen+1):
                tmp+=mod[max(0,min(self._nd-1,ib))]
            dat[i]+=tmp/sc


    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()
        
        sc=1./(1+2.*self._halflen)
        for i in range(self._nd):
            tmp=0
            for ib in range(i-self._halflen,i+self._halflen+1):
                mod[max(0,min(self._nd-1,ib))]+=dat[i]/sc
    


In [32]:
from sep_python import get_sep_vector
import numpy as np
vec=get_sep_vector(np.zeros((30,),dtype=np.float32))
box_op=BoxcarF(vec,vec,10)
test_dot(box_op)

-387.22397 -387.22394
